In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
from metpy.constants import Rd
from metpy.units import units, concatenate
%matplotlib inline

In [2]:
df = pd.read_csv('Soundings_1992.txt', index_col=0)

/Users/johnleeman/miniconda3/envs/metpydev/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df.head()

,Date,Time,Station,Pressure,Temperature,Dewpoint,u wind,v wind,Speed,Direction
0,1992-01-01,0,UIL,1012.0,NaN,NaN,-4.000000,6.928203,8.0,150.0
1,1992-01-01,0,UIL,1000.0,10.2,3.2,-5.916656,12.688309,14.0,155.0
2,1992-01-01,0,UIL,982.9,9.3,2.9,-7.182423,19.733545,21.0,160.0
3,1992-01-01,0,UIL,947.4,7.3,2.3,-5.556742,31.513848,32.0,170.0
4,1992-01-01,0,UIL,913.2,5.4,1.8,-3.573385,40.843983,41.0,175.0


In [4]:
def parcel_profile_tv(p, T, Td):
    prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')
    #Find dewpoint for the virtual temperature calculation for the parcel
    #Below LCL: dewpoint = environmental dewpoint
    #Above LCL: dewpoint = parcel temp, since it's saturated.
    lcl_pressure,_ = mpcalc.lcl(p[0], T[0], Td[0])
    parcel_dewpoint = Td
    idx = np.where(p <= lcl_pressure)
    parcel_dewpoint[idx] = T[idx]
    
    Rv = mpcalc.mixing_ratio(mpcalc.saturation_vapor_pressure(parcel_dewpoint), p)
    return  mpcalc.virtual_temperature(prof, Rv)

In [5]:
def most_unstable_profile(p, T, Td):
    most_unstable = mpcalc.most_unstable_parcel(p, T, Td)
    idx = p <= most_unstable[0]
    return p[idx], T[idx], Td[idx]

In [6]:
calculations = []

for name, group in df.groupby(['Date', 'Time', 'Station']):
    print(name)
    
    # Pull out data and put units back on it
    p = np.array(group['Pressure']) * units.hPa
    T = np.array(group['Temperature']) * units.degC 
    Td = np.array(group['Dewpoint']) * units.degC
    
    # Make limit data to most unstable region
    try:
        p, T, Td = most_unstable_profile(p, T, Td)
    except:
        print("Can't get most unstable parcel.")
        continue
    
    # CAPE/CIN with no virtual temperature correction
    try:
        parcel_path = mpcalc.parcel_profile(p, T[0], Td[0])
        cape, cin= mpcalc.cape_cin(p, T, Td, parcel_path)
        cape = cape.m
        cin = cin.m
    except:
        cape = 0
        cin = 0
        print("ERROR IN CAPE/CIN", name)
    #cape, cin= mpcalc.cape_cin(p, T, Td, parcel_path)    
    # CAPE/CIN with virtual temperature correction
    try:
        parcel_path = parcel_profile_tv(p, T, Td)
        cape_tv, cin_tv= mpcalc.cape_cin(p, T, Td, parcel_path)
        cape_tv = cape_tv.m
        cin_tv = cin_tv.m
    except:
        cape_tv = 0
        cin_tv = 0
        print("ERROR IN CAPE/CIN TV", name)
    
    calculations.append({'Date': name[0], 
                         'Time': name[1], 
                         'Station': name[2], 
                         'CAPE': cape, 
                         'CIN': cin, 
                         'CAPE_Tv': cape_tv,
                         'CIN_Tv': cin_tv})

('1992-01-01', 0, 'ABQ')
('1992-01-01', 0, 'ALB')
('1992-01-01', 0, 'AMA')
Can't get most unstable parcel.
('1992-01-01', 0, 'BIS')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'BIS')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'BIS')
('1992-01-01', 0, 'BNA')
('1992-01-01', 0, 'BOI')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'BOI')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'BOI')
('1992-01-01', 0, 'BUF')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'BUF')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'BUF')
('1992-01-01', 0, 'CAR')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'CAR')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'CAR')
('1992-01-01', 0, 'CHH')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'CHH')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'CHH')
('1992-01-01', 0, 'CHS')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'CHS')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'CHS')
('1992-01-01', 0, 'CRP')
('1992-01-01', 0, 'DDC')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'DDC')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'DDC')
('1992-01-01', 0, 'DNR')
ERROR IN CAPE/CIN ('

/Users/johnleeman/miniconda3/envs/metpydev/lib/python3.6/site-packages/pint/quantity.py:1023: RuntimeWarning: invalid value encountered in double_scalars
  return other**new_self._magnitude


('1992-01-01', 0, 'SLC')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'SLC')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'SLC')
('1992-01-01', 0, 'TBW')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'TBW')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'TBW')
('1992-01-01', 0, 'TLH')
('1992-01-01', 0, 'TOP')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'TOP')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'TOP')
('1992-01-01', 0, 'TUS')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'TUS')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'TUS')
('1992-01-01', 0, 'UIL')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'UIL')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'UIL')
('1992-01-01', 0, 'WAL')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'WAL')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'WAL')
('1992-01-01', 0, 'WMW')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'WMW')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'WMW')
('1992-01-01', 0, 'WPL')
('1992-01-01', 0, 'YMO')
ERROR IN CAPE/CIN ('1992-01-01', 0, 'YMO')
ERROR IN CAPE/CIN TV ('1992-01-01', 0, 'YMO')
('1992-01-01', 0, 'YQI')
ERROR IN CAPE

/Users/johnleeman/gitrepos/MetPy/metpy/calc/thermo.py:995: RuntimeWarning: invalid value encountered in greater
  keep_idx = np.ediff1d(x, to_end=[1]) > 0
/Users/johnleeman/miniconda3/envs/metpydev/lib/python3.6/site-packages/pint/quantity.py:1065: RuntimeWarning: invalid value encountered in less_equal
  return op(self._convert_magnitude_not_inplace(UnitsContainer()), other)
/Users/johnleeman/miniconda3/envs/metpydev/lib/python3.6/site-packages/pint/quantity.py:1065: RuntimeWarning: invalid value encountered in greater_equal
  return op(self._convert_magnitude_not_inplace(UnitsContainer()), other)
/Users/johnleeman/miniconda3/envs/metpydev/lib/python3.6/site-packages/pint/quantity.py:1070: RuntimeWarning: invalid value encountered in less_equal
  return op(self._magnitude, other._magnitude)


('1992-01-01', 12, 'BNA')
ERROR IN CAPE/CIN ('1992-01-01', 12, 'BNA')
ERROR IN CAPE/CIN TV ('1992-01-01', 12, 'BNA')
('1992-01-01', 12, 'BOI')
('1992-01-01', 12, 'BRO')
('1992-01-01', 12, 'BUF')
ERROR IN CAPE/CIN ('1992-01-01', 12, 'BUF')
ERROR IN CAPE/CIN TV ('1992-01-01', 12, 'BUF')
('1992-01-01', 12, 'CAR')
ERROR IN CAPE/CIN ('1992-01-01', 12, 'CAR')
ERROR IN CAPE/CIN TV ('1992-01-01', 12, 'CAR')
('1992-01-01', 12, 'CHH')
('1992-01-01', 12, 'CHS')
('1992-01-01', 12, 'CRP')
ERROR IN CAPE/CIN ('1992-01-01', 12, 'CRP')
ERROR IN CAPE/CIN TV ('1992-01-01', 12, 'CRP')
('1992-01-01', 12, 'DDC')


...

('1992-12-15', 0, 'TBW')
('1992-12-15', 0, 'TLH')
('1992-12-15', 0, 'TOP')
('1992-12-15', 0, 'TUS')
('1992-12-15', 0, 'UIL')
('1992-12-15', 0, 'WAL')
('1992-12-15', 0, 'WMW')
('1992-12-15', 0, 'YMO')
('1992-12-15', 0, 'YQI')
('1992-12-15', 12, 'ABQ')
('1992-12-15', 12, 'ADN')
('1992-12-15', 12, 'ALB')
('1992-12-15', 12, 'AMA')
('1992-12-15', 12, 'BIS')
('1992-12-15', 12, 'BNA')
('1992-12-15', 12, 'BOI')
('1992-12-15', 12, 'BRO')
('1992-12-15', 12, 'BUF')
('1992-12-15', 12, 'CAR')
('1992-12-15', 12, 'CHH')
('1992-12-15', 12, 'CHS')
('1992-12-15', 12, 'CRP')
('1992-12-15', 12, 'DDC')
('1992-12-15', 12, 'DNR')
('1992-12-15', 12, 'DRT')
('1992-12-15', 12, 'GGW')
('1992-12-15', 12, 'GJT')
('1992-12-15', 12, 'GRB')
('1992-12-15', 12, 'GSO')
('1992-12-15', 12, 'IAD')
('1992-12-15', 12, 'INL')
('1992-12-15', 12, 'JAN')
('1992-12-15', 12, 'LBF')
('1992-12-15', 12, 'LCH')
('1992-12-15', 12, 'LIX')
('1992-12-15', 12, 'LZK')
('1992-12-15', 12, 'MAF')
('1992-12-15', 12, 'MFR')
('1992-12-15', 12, 'N

('1992-12-19', 0, 'AMA')
('1992-12-19', 0, 'BIS')
('1992-12-19', 0, 'BNA')
('1992-12-19', 0, 'BOI')
('1992-12-19', 0, 'BRO')
('1992-12-19', 0, 'BUF')
('1992-12-19', 0, 'CAR')
('1992-12-19', 0, 'CHH')
('1992-12-19', 0, 'CHS')
('1992-12-19', 0, 'CRP')
('1992-12-19', 0, 'DDC')
('1992-12-19', 0, 'DNR')
('1992-12-19', 0, 'DRT')
('1992-12-19', 0, 'GGW')
('1992-12-19', 0, 'GJT')
('1992-12-19', 0, 'GRB')
('1992-12-19', 0, 'GSO')
('1992-12-19', 0, 'IAD')
('1992-12-19', 0, 'INL')
('1992-12-19', 0, 'JAN')
('1992-12-19', 0, 'LBF')
('1992-12-19', 0, 'LCH')
('1992-12-19', 0, 'LIX')
('1992-12-19', 0, 'LZK')
('1992-12-19', 0, 'MAF')
('1992-12-19', 0, 'MFR')
('1992-12-19', 0, 'NKX')
('1992-12-19', 0, 'OAK')
('1992-12-19', 0, 'OUN')
('1992-12-19', 0, 'PIT')
('1992-12-19', 0, 'RAP')
('1992-12-19', 0, 'SLC')
('1992-12-19', 0, 'TBW')
('1992-12-19', 0, 'TLH')
('1992-12-19', 0, 'TOP')
('1992-12-19', 0, 'TUS')
('1992-12-19', 0, 'UIL')
('1992-12-19', 0, 'WAL')
('1992-12-19', 0, 'WMW')
('1992-12-19', 0, 'WPL')


('1992-12-22', 12, 'DDC')
('1992-12-22', 12, 'DNR')
('1992-12-22', 12, 'DRT')
('1992-12-22', 12, 'GGW')
('1992-12-22', 12, 'GJT')
('1992-12-22', 12, 'GRB')
('1992-12-22', 12, 'GSO')
('1992-12-22', 12, 'INL')
('1992-12-22', 12, 'JAN')
('1992-12-22', 12, 'LBF')
('1992-12-22', 12, 'LCH')
('1992-12-22', 12, 'LIX')
('1992-12-22', 12, 'LZK')
('1992-12-22', 12, 'MAF')
('1992-12-22', 12, 'MFR')
('1992-12-22', 12, 'NKX')
('1992-12-22', 12, 'OAK')
('1992-12-22', 12, 'OUN')
('1992-12-22', 12, 'PIT')
('1992-12-22', 12, 'RAP')
('1992-12-22', 12, 'SLC')
('1992-12-22', 12, 'TBW')
('1992-12-22', 12, 'TLH')
('1992-12-22', 12, 'TOP')
('1992-12-22', 12, 'TUS')
('1992-12-22', 12, 'UIL')
('1992-12-22', 12, 'WAL')
('1992-12-22', 12, 'WMW')
('1992-12-22', 12, 'WPL')
('1992-12-22', 12, 'YMO')
('1992-12-23', 0, 'ABQ')
('1992-12-23', 0, 'ALB')
('1992-12-23', 0, 'AMA')
('1992-12-23', 0, 'BIS')
('1992-12-23', 0, 'BNA')
('1992-12-23', 0, 'BOI')
('1992-12-23', 0, 'BRO')
('1992-12-23', 0, 'BUF')
('1992-12-23', 0, 'C

('1992-12-26', 0, 'TOP')
('1992-12-26', 0, 'TUS')
('1992-12-26', 0, 'UIL')
('1992-12-26', 0, 'WAL')
('1992-12-26', 0, 'WMW')
('1992-12-26', 0, 'WPL')
('1992-12-26', 0, 'YMO')
('1992-12-26', 0, 'YQI')
('1992-12-26', 12, 'ABQ')
('1992-12-26', 12, 'ADN')
('1992-12-26', 12, 'ALB')
('1992-12-26', 12, 'AMA')
('1992-12-26', 12, 'BIS')
('1992-12-26', 12, 'BNA')
('1992-12-26', 12, 'BOI')
('1992-12-26', 12, 'BRO')
('1992-12-26', 12, 'BUF')
('1992-12-26', 12, 'CAR')
('1992-12-26', 12, 'CHH')
('1992-12-26', 12, 'CHS')
('1992-12-26', 12, 'CRP')
('1992-12-26', 12, 'DDC')
('1992-12-26', 12, 'DNR')
('1992-12-26', 12, 'DRT')
('1992-12-26', 12, 'GGW')
('1992-12-26', 12, 'GJT')
('1992-12-26', 12, 'GRB')
('1992-12-26', 12, 'IAD')
('1992-12-26', 12, 'INL')
('1992-12-26', 12, 'JAN')
('1992-12-26', 12, 'LBF')
('1992-12-26', 12, 'LCH')
('1992-12-26', 12, 'LIX')
('1992-12-26', 12, 'LZK')
('1992-12-26', 12, 'MAF')
('1992-12-26', 12, 'MFR')
('1992-12-26', 12, 'NKX')
('1992-12-26', 12, 'OAK')
('1992-12-26', 12, '

('1992-12-30', 0, 'BUF')
('1992-12-30', 0, 'CAR')
('1992-12-30', 0, 'CHH')
('1992-12-30', 0, 'CHS')
('1992-12-30', 0, 'CRP')
('1992-12-30', 0, 'DDC')
('1992-12-30', 0, 'DNR')
('1992-12-30', 0, 'DRT')
('1992-12-30', 0, 'GGW')
('1992-12-30', 0, 'GJT')
('1992-12-30', 0, 'GRB')
('1992-12-30', 0, 'GSO')
('1992-12-30', 0, 'IAD')
('1992-12-30', 0, 'INL')
('1992-12-30', 0, 'JAN')
('1992-12-30', 0, 'LBF')
('1992-12-30', 0, 'LCH')
('1992-12-30', 0, 'LIX')
('1992-12-30', 0, 'LZK')
('1992-12-30', 0, 'MAF')
('1992-12-30', 0, 'MFR')
('1992-12-30', 0, 'NKX')
('1992-12-30', 0, 'OAK')
('1992-12-30', 0, 'OUN')
('1992-12-30', 0, 'PIT')
('1992-12-30', 0, 'RAP')
('1992-12-30', 0, 'SLC')
('1992-12-30', 0, 'TBW')
('1992-12-30', 0, 'TLH')
('1992-12-30', 0, 'TOP')
('1992-12-30', 0, 'TUS')
('1992-12-30', 0, 'UIL')
('1992-12-30', 0, 'WAL')
('1992-12-30', 0, 'WMW')
('1992-12-30', 0, 'WPL')
('1992-12-30', 0, 'YMO')
('1992-12-30', 0, 'YQI')
('1992-12-30', 12, 'ABQ')
('1992-12-30', 12, 'ADN')
('1992-12-30', 12, 'ALB

In [7]:
# Empty dataframe to hold the results of our calculations
#calc_df = pd.DataFrame(columns=['Date', 'Time', 'Station', 'CAPE', 'CIN', 'CAPE_Tv', 'CIN_Tv'])

# Put our list of results into the dataframe
calc_df = pd.DataFrame(calculations)

In [8]:
calc_df.head()

,CAPE,CAPE_Tv,CIN,CIN_Tv,Date,Station,Time
0,0.0,0.0,-54.190162,-51.281115,1992-01-01,ABQ,0
1,0.0,0.0,0.000000,0.000000,1992-01-01,ALB,0
2,0.0,0.0,0.000000,0.000000,1992-01-01,BIS,0
3,0.0,0.0,-599.520288,-596.857519,1992-01-01,BNA,0
4,0.0,0.0,0.000000,0.000000,1992-01-01,BOI,0


In [9]:
calc_df.to_csv('1992_Calculations.txt')